In [11]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import metatensor
from metatensor import Labels, TensorBlock, TensorMap
import chemiscope

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import old_clebsch_gordan
import rotations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate a rascaline SphericalExpansion calculator

In [2]:
# frames = ase.io.read("frame.xyz", ":")
frames = ase.io.read("combined_magres_spherical.xyz", ":5")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 4,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}
calculator = rascaline.SphericalExpansion(**rascal_hypers)

# Equivariance Test - SO(3) for $\nu=3$

In [38]:
# Define target angular channels
angular_selection = [0, 2]

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Randomly rigidly rotate the frame
frames_so3 = [rotations.transform_frame_so3(frame, wig.angles) for frame in frames]
assert not np.allclose(frames[-1].positions, frames_so3[-1].positions)

# Generate nu=3 descriptor for both frames
print("Computing nu = 3 descriptor for unrotated frames...")
nu_1 = calculator.compute(frames)
nu_3 = clebsch_gordan.combine_single_center_to_body_order(
    nu_1_tensor=nu_1,
    target_body_order=3,
    angular_selection=angular_selection,
    parity_selection=[+1],
)
print("Computing nu = 3 descriptor for rotated frames...")
nu_1_rot = calculator.compute(frames_so3)
nu_3_rot = clebsch_gordan.combine_single_center_to_body_order(
    nu_1_tensor=nu_1_rot,
    target_body_order=3,
    angular_selection=angular_selection,
    parity_selection=[+1],
)

# Rotate the lambda-SOAP descriptor of the unrotated frame
nu_3_transf = wig.transform_tensormap_so3(nu_3)

# Check for equivariance!
assert metatensor.equal_metadata(nu_3_transf, nu_3_rot)
assert metatensor.allclose(nu_3_transf, nu_3_rot)
print("SO(3) EQUIVARIANT!")

# chemiscope.show(frames + frames_so3, mode="structure")

Random rotation angles (rad): [5.77507977 5.81759295 0.3318965 ]
Computing nu = 3 descriptor for unrotated frames...
Computing nu = 3 descriptor for rotated frames...
SO(3) EQUIVARIANT!


# Equivariance Test - O(3) on $\nu=2$ (i.e. $\lambda$-SOAP)

In [39]:
# Define target lambda channels
angular_selection = [0, 1, 2, 3, 4, 5]

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Apply an O(3) transformation to the frame
frames_o3 = [rotations.transform_frame_o3(frame, wig.angles) for frame in frames]
assert not np.allclose(frames[-1].positions, frames_o3[-1].positions)

# Generate lambda-SOAP for both frames
print("Computing lambda-SOAP descriptor for unrotated frames...")
nu_1 = calculator.compute(frames)
lsoap = clebsch_gordan.lambda_soap_vector(
    nu_1_tensor=nu_1,
    angular_selection=angular_selection,
)
print("Computing lambda-SOAP descriptor for rotated frames...")
nu_1_rot = calculator.compute(frames_o3)
lsoap_o3 = clebsch_gordan.lambda_soap_vector(
    nu_1_tensor=nu_1_rot,
    angular_selection=angular_selection,
)

# Apply the O(3) transformation to the TensorMap
lsoap_transf = wig.transform_tensormap_o3(lsoap)

# Check for equivariance!
assert metatensor.equal_metadata(lsoap_transf, lsoap_o3)
assert metatensor.allclose(lsoap_transf, lsoap_o3)
print("O(3) EQUIVARIANT!")

# chemiscope.show(frames + frames_o3, mode="structure")

Random rotation angles (rad): [2.72689356 0.10058627 6.14346159]
Computing lambda-SOAP descriptor for unrotated frames...
Computing lambda-SOAP descriptor for rotated frames...
O(3) EQUIVARIANT!


# Old v new lambda-SOAP

In [26]:
%%timeit
lsoap_old = old_clebsch_gordan.lambda_soap_vector(frames, rascal_hypers, lambda_cut=5)

size = 0
for block in lsoap_old:
    size += np.prod(block.values.shape)
print("Size:", size)

Size: 23315040
Size: 23315040
Size: 23315040
Size: 23315040
Size: 23315040
Size: 23315040
Size: 23315040
Size: 23315040
731 ms ± 9.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
calculator = rascaline.SphericalExpansion(**rascal_hypers)
nu_1 = calculator.compute(frames)
lsoap_new = clebsch_gordan.lambda_soap_vector(nu_1, angular_cutoff=5)

size = 0
for block in lsoap_new:
    size += np.prod(block.values.shape)
print("Size:", size)

Size: 13819680
Size: 13819680
Size: 13819680
Size: 13819680
Size: 13819680
Size: 13819680
Size: 13819680
Size: 13819680
410 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Dense v Sparse

In [34]:
# Dense
calculator = rascaline.SphericalExpansion(**rascal_hypers)
nu_1 = calculator.compute(frames[:2])
nu_2_dense = clebsch_gordan.combine_single_center_to_body_order(
    nu_1,
    target_body_order=3,
    angular_cutoff=5,
    use_sparse=False,
)
nu_2_dense

TensorMap with 36 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         3             1                   0                  3
         3            -1                   1                  3
                                    ...
         3             1                   5                  22
         3            -1                   5                  22

In [36]:
# Sparse
calculator = rascaline.SphericalExpansion(**rascal_hypers)
nu_1 = calculator.compute(frames[:2])
nu_2_sparse = clebsch_gordan.combine_single_center_to_body_order(
    nu_1,
    target_body_order=3,
    angular_cutoff=5,
    use_sparse=True,
)
nu_2_sparse

TensorMap with 36 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         3             1                   0                  3
         3            -1                   1                  3
                                    ...
         3             1                   5                  22
         3            -1                   5                  22

In [37]:
# Check sparse == dense
metatensor.allclose(nu_2_dense, nu_2_sparse)

True

# Test system 1

In [ ]:
# %%timeit

# use_sparse = False

# tensor_dense = clebsch_gordan.n_body_iteration_single_center(
#     frames,
#     rascal_hypers=rascal_hypers,
#     nu_target=3,
#     lambdas=lambdas,
#     use_sparse=use_sparse,
# )
# tensor_dense

# # timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
# %%timeit

# use_sparse = True

# tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
#     frames,
#     rascal_hypers=rascal_hypers,
#     nu_target=3,
#     lambdas=lambdas,
#     use_sparse=use_sparse,
# )
# tensor_sparse

# # timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
# tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
#     frames,
#     rascal_hypers=rascal_hypers,
#     nu_target=3,
#     lambdas=lambdas,
#     use_sparse=True,
# )

# tensor_dense = clebsch_gordan.n_body_iteration_single_center(
#     frames,
#     rascal_hypers=rascal_hypers,
#     nu_target=3,
#     lambdas=lambdas,
#     use_sparse=False,
# )

# assert metatensor.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
# frames = [ase.io.read("frame.xyz")]

# lambdas = np.array([0, 2])

# rascal_hypers = {
#     "cutoff": 3.0,  # Angstrom
#     "max_radial": 6,  # Exclusive
#     "max_angular": 5,  # Inclusive
#     "atomic_gaussian_width": 0.2,
#     "radial_basis": {"Gto": {}},
#     "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
#     "center_atom_weight": 1.0,
# }

# n_body = clebsch_gordan.n_body_iteration_single_center(
#     frames,
#     rascal_hypers=rascal_hypers,
#     nu_target=3,
#     lambdas=lambdas,
# )
# n_body

In [ ]:
# def sort_tm(tm):
#     blocks = []
#     for _, block in tm.items():
#         values = block.values

#         samples_values = block.samples.values
#         sorted_idx = native_list_argsort([tuple(row.tolist()) for row in block.samples.values])
#         samples_values = samples_values[sorted_idx]
#         values = values[sorted_idx]

#         components_values = []
#         for i, component in enumerate(block.components):
#             component_values = component.values
#             sorted_idx = native_list_argsort([tuple(row.tolist()) for row in component.values])
#             components_values.append( component_values[sorted_idx] )
#             values = np.take(values, sorted_idx, axis=i+1)

#         properties_values = block.properties.values
#         sorted_idx = native_list_argsort([tuple(row.tolist()) for row in block.properties.values])
#         properties_values = properties_values[sorted_idx]
#         values = values[..., sorted_idx]

#         blocks.append(
#             TensorBlock(
#                 values=values,
#                 samples=Labels(values=samples_values, names=block.samples.names),
#                 components=[Labels(values=components_values[i], names=component.names) for i, component in enumerate(block.components)],
#                 properties=Labels(values=properties_values, names=block.properties.names)
#             )
#         )
#     return TensorMap(keys=tm.keys, blocks=blocks)


# def native_list_argsort(native_list):
#     return sorted(range(len(native_list)), key=native_list.__getitem__)


In [ ]:

# # Manipulate metadata from old LSOAP
# lsoap_old = metatensor.permute_dimensions(lsoap_old0, axis="properties", dimensions_indexes=[2, 5, 0, 1, 3, 4])
# lsoap_old = metatensor.rename_dimension(lsoap_old, axis="properties", old="l_1", new="l1")
# lsoap_old = metatensor.rename_dimension(lsoap_old, axis="properties", old="l_2", new="l2")
# lsoap_old = metatensor.rename_dimension(lsoap_old, axis="properties", old="n_1", new="n1")
# lsoap_old = metatensor.rename_dimension(lsoap_old, axis="properties", old="n_2", new="n2")

# # Slice TM to symmetrize l-values
# sliced_blocks = []
# for key, block in lsoap_old.items():
#     # Filter properties l1 <= l2
#     mask = [entry[0] <= entry[1] for entry in block.properties]
#     new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

#     # Slice block
#     sliced_block = metatensor.slice_block(block, axis="properties", labels=new_labels)
#     sliced_blocks.append(sliced_block)

# # Check equal metadata
# lsoap_old = sort_tm(TensorMap(keys=lsoap_old.keys, blocks=sliced_blocks))
# lsoap_new = sort_tm(lsoap_new0)
# assert metatensor.equal_metadata(lsoap_old, lsoap_new)

# # Check equal values
# metatensor.allclose_raise(lsoap_old, lsoap_new)

In [ ]:
# for key in lsoap_old.keys:
#     b1 = lsoap_old[key]
#     b2 = lsoap_new[key]

#     print(key, metatensor.allclose_block(b1, b2))

In [ ]:
# np.linalg.norm(
#     lsoap_old.block(inversion_sigma=1, spherical_harmonics_l=1, species_center=3).values
#     - lsoap_new.block(
#         inversion_sigma=1, spherical_harmonics_l=1, species_center=3
#     ).values
# )

In [ ]:
# # NOW SLICE TO l1 == l2

# # Slice TM
# sliced_blocks = []
# for key, block in lsoap_old.items():
#     # Filter properties
#     mask = [entry[0] == entry[1] for entry in block.properties]
#     new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

#     # Slice block
#     sliced_blocks.append(metatensor.slice_block(block, axis="properties", labels=new_labels))

# lsoap_old_sliced = TensorMap(keys=lsoap_old.keys, blocks=sliced_blocks)

# # Slice TM
# sliced_blocks = []
# for key, block in lsoap_new.items():
#     # Filter properties
#     mask = [entry[0] == entry[1] for entry in block.properties]
#     new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

#     # Slice block
#     sliced_blocks.append(metatensor.slice_block(block, axis="properties", labels=new_labels))

# lsoap_new_sliced = TensorMap(keys=lsoap_new.keys, blocks=sliced_blocks)

# assert metatensor.equal_metadata(lsoap_old_sliced, lsoap_new_sliced)
# assert metatensor.allclose_raise(lsoap_old_sliced, lsoap_new_sliced)